In [ ]:
import numpy as np
import os
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt

train_dir = '../input/asl_alphabet_train/asl_alphabet_train'
test_dir = '../input/asl_alphabet_test/asl_alphabet_test'

In [ ]:
img_size = 64 

labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}

# directory should be the path to the folder with the images, use the predefined variables 'train' and 'evaluation'
# The size is for resizing the images.  All images are squares.  The reason we decided to implement this size variable is because
# we found multiple different proposed solutions using image sizes ranging from 64 to 580, with such a large range we wanted room to 
# adjust later in the process.
def load_data():
    print("loading images")
    images = []
    labels = []
    # size = 64,64
    # print("LOADING DATA FROM : ",end = "")
    for folder in os.listdir(train_dir):
        # print(folder, end = ' | ')
        for image in os.listdir(train_dir + "/" + folder):
            img = cv2.imread(train_dir + '/' + folder + '/' + image)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(labels_dict[folder])
    
    images = np.array(images)
    images = images.astype('float32')/255.0
    
    labels = keras.utils.to_categorical(labels)
    
    return images, labels

images, labels = load_data()

# Here we are going to use scikit to split our training data into two separate pieces, training and testing (note testing is separate from evaluation)
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.1)

# This is to load our evaluation data for the final deliverable
# if labels_set == sorted(os.listdir(evaluation)):
#     x_eval, y_eval = load_images(directory = evaluation, size = img_size)

print('Loaded', len(X_train),'images for training,')
print('Loaded', len(X_test),'images for testing')

In [ ]:
def create_model():

    kernel = [3,3]
    
    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = kernel, padding = 'same', activation = 'relu', input_shape = (64,64,3)))
    model.add(Conv2D(32, kernel_size = kernel, padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = kernel))
    
    model.add(Conv2D(32, kernel_size = kernel, padding = 'same', activation = 'relu'))
    model.add(Conv2D(64, kernel_size = kernel, padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = kernel))
    
    model.add(Conv2D(128, kernel_size = kernel, padding = 'same', activation = 'relu'))
    model.add(Conv2D(256, kernel_size = kernel, padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = kernel))
    
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(29, activation = 'softmax'))
    
    model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ["accuracy"])
    
    print("MODEL CREATED")
    model.summary()
    
    return model

def fit_model():
    model_hist = model.fit(X_train, Y_train, batch_size = 64, epochs = 5, validation_split = 0.1)
    return model_hist 

model = create_model()
curr_model_hist = fit_model()

In [ ]:
plt.plot(curr_model_hist.history['acc'])
plt.plot(curr_model_hist.history['val_acc'])
plt.legend(['train', 'test'], loc='lower right')
plt.title('accuracy plot - train vs test')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

plt.plot(curr_model_hist.history['loss'])
plt.plot(curr_model_hist.history['val_loss'])
plt.legend(['training loss', 'validation loss'], loc = 'upper right')
plt.title('loss plot - training vs vaidation')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

evaluate_metrics = model.evaluate(X_test, Y_test)
print("\nEvaluation Accuracy = ", "{:.2f}%".format(evaluate_metrics[1]*100),"\nEvaluation loss = " ,"{:.6f}".format(evaluate_metrics[0]))

In [ ]:
def load_test_data():
    images = []
    names = []
    # size = 64,64
    for image in os.listdir(test_dir):
        temp = cv2.imread(test_dir + '/' + image)
        temp = cv2.resize(temp, (img_size, img_size))
        images.append(temp)
        names.append(image)
    images = np.array(images)
    images = images.astype('float32')/255.0
    return images, names

def get_labels_for_plot(predictions):
    predictions_labels = []
    for i in range(len(predictions)):
        for ins in labels_dict:
            if predictions[i] == labels_dict[ins]:
                predictions_labels.append(ins)
                break
    return predictions_labels

predictions_labels_plot = get_labels_for_plot(predictions)

test_images, test_img_names = load_test_data()

# make predictions on an image and append it to the list (predictions).
predictions = [model.predict_classes(image.reshape(1,64,64,3))[0] for image in test_images]


In [ ]:
predfigure = plt.figure(figsize = (13,13))
def plot_image_1(fig, image, label, prediction, predictions_label, row, col, index):
    fig.add_subplot(row, col, index)
    plt.axis('off')
    plt.imshow(image)
    title = "prediction : [" + str(predictions_label) + "] "+ "\n" + label
    plt.title(title)
    return

image_index = 0
row = 5
col = 6
for i in range(1,(row*col-1)):
    plot_image_1(predfigure, test_images[image_index], test_img_names[image_index], predictions[image_index], predictions_labels_plot[image_index], row, col, i)
    image_index = image_index + 1
plt.show()



In [ ]:
model.save('asl_predictor.h5')